In [1]:
import numpy as np
import pandas as pd
#for reading in data properly
import ast
import json

import gensim
from gensim.models import Word2Vec

from sklearn.model_selection import train_test_split
from sklearn import utils
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

import re

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import keras.backend as K
from keras.callbacks import EarlyStopping
from keras.layers import Conv1D, GlobalMaxPooling1D, LSTM, SimpleRNN, Dense, Dropout, Flatten, Bidirectional
from keras.layers import Input, concatenate, Activation
from keras.layers.embeddings import Embedding
from keras.models import Sequential, Model
from keras.regularizers import l2

stop_words = set(stopwords.words('english'))

import time

Using TensorFlow backend.


read in the data

In [2]:
all_data = pd.read_csv('train.csv')
all_data = all_data.dropna(subset=['overview', 'genres']) #drop cols without overview or genre (data we use or labels)

In [3]:
#parse each row to get label vectors from json
def parse_genres_json(x):
    try:
        json_genres = json.loads(x.replace("'", '"'))
        numElems = len(json_genres)
        ret = [0]*len(genre_dict) #number of genres we are looking at
        for i in range(numElems):
            genre_str = (json_genres[i]['name'])
            if genre_str in genre_map.keys():
                ret[genre_dict[genre_map[genre_str]]] = 1
        return ret
    except Exception as excep:
        print('Exception' + str(excep))
        return ''

Get dictionary for genre to its index in label vector

In [4]:
#incldue ALL genres of data set in genre dict as is
genre_dict = {'War': 0,
 'Family': 1,
 'Science Fiction': 2,
 'Thriller': 3,
 'Horror': 4,
 'Romance': 5,
 'Drama': 6,
 'Foreign': 7,
 'Documentary': 8,
 'Fantasy': 9,
 'Western': 10,
 'History': 11,
 'Comedy': 12,
 'Action': 13,
 'Adventure': 14,
 'Animation': 15,
 'Crime': 16,
 'Music': 17,
 'TV Movie': 18,
 'Mystery': 19}

In [5]:
#for mapping to coarse grained labels (in this situation we don't do that so labels map to self)
#maps to self here as we don't do anything special with labels in this file
genre_map = {'War': 'War',
 'Family': 'Family',
 'Science Fiction': 'Science Fiction',
 'Thriller': 'Thriller',
 'Horror': 'Horror',
 'Romance': 'Romance',
 'Drama': 'Drama',
 'Foreign': 'Foreign',
 'Documentary': 'Documentary',
 'Fantasy': 'Fantasy',
 'Western': 'Western',
 'History': 'History',
 'Comedy': 'Comedy',
 'Action': 'Action',
 'Adventure': 'Adventure',
 'Animation': 'Animation',
 'Crime': 'Crime',
 'Music': 'Music',
 'TV Movie': 'TV Movie',
 'Mystery': 'Mystery'}

In [6]:
def getGenresVects():
    y = all_data['genres']
    ret = y.apply(parse_genres_json)
    all_data['genres_vect'] = ret

In [7]:
getGenresVects() #get label vectors for genres indexed by indexes in genre_dict

In [8]:
#put to lower case, remove punctation, remove stopwords
def cleanText(text):
    no_stopword_text = [w for w in text.split() if not w in stop_words]
    text = ' '.join(no_stopword_text)
    text = re.sub(r'[^a-z A-Z0-9]', "", text) #maybe shouldn't remove punction between words here?
    text = text.lower()
    return text

all_data['cleanOverview'] = all_data['overview'].apply(cleanText)

In [9]:
all_data = all_data[all_data.genres_vect.map(sum) > 0] #drop rows that now have no labels 

In [10]:
#neural net data only needs a few cols
nn_data = all_data[['cleanOverview', 'genres_vect', 'overview']]

In [11]:
train, test = train_test_split(nn_data, test_size=0.2, random_state=42)

Extract actual features and labels from train and test set

In [12]:
#gettrian and test features for classification. Just need text and lables for this
x = train['cleanOverview'].values.tolist()
y = train['genres_vect']
x_test = test['cleanOverview'].values.tolist()
y_test = test['genres_vect']

In [13]:
#convert labels from array of lists to numpy array

y_train = y.tolist()
y_train = np.array(y_train)

y_test = y_test.tolist()
y_test = np.array(y_test)

Get initial word embedding vectors

In [14]:
tok = [word_tokenize(ov) for ov in x]

In [15]:
word_vec_len = 32
w2v = Word2Vec(tok, min_count = 2, size=word_vec_len)

In [16]:
num_words_kept = 100000 #using 100000 most popular words, use throughout

tokenizer = Tokenizer(num_words_kept)
tokenizer.fit_on_texts(x)
sequences = tokenizer.texts_to_sequences(x)

max_seq_len = 150 #larger than averaage but not too large

#get actual train features to feed into neural nets for training
x_train_seq = pad_sequences(sequences, maxlen=max_seq_len)

In [17]:
test_sequences = tokenizer.texts_to_sequences(x_test)
#get actual test features to feed into neural nets for testing
x_test_seq = pad_sequences(test_sequences, maxlen=max_seq_len)

Get word embeddings matrix for start input to neural net

In [18]:
#Citation: This technique to get word embeddings comes, with some minor changes, mostly from: 
#https://towardsdatascience.com/another-twitter-sentiment-analysis-with-python-part-11-cnn-word2vec-41f5e28eda74

embeddings_index = {}
for w in w2v.wv.vocab.keys():
    embeddings_index[w] = w2v.wv[w]


embedding_matrix = np.zeros((num_words_kept, word_vec_len))
for word, i in tokenizer.word_index.items():
    if i >= num_words_kept:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

Below we define evlaution metric functions

In [19]:
def get_per_label_metrics(real_labels_matrix, predictions_labels_matrix):
    for genre in genre_dict.keys():
        index = genre_dict[genre]
        real_labels_vect = real_labels_matrix[:, index]
        prediction_vect = predictions_labels_matrix[:,index]
        print("Accuruacy for " + genre + ": " + str(accuracy_score(real_labels_vect, prediction_vect)))
        print("Precision for " + genre + ": " + str(precision_score(real_labels_vect, prediction_vect)))
        print("Recall for " + genre + ": " + str(recall_score(real_labels_vect, prediction_vect)))
        print()

In [20]:
#size of intersection of predicted and actual labels divided by size of their union for each datapoint tested on
#sum those and then divide by number of datapoints
#vectorized for speed
def multi_label_accuracy(real_labels_matrix, predictions_labels_matrix):
    #binary so set intersection is and operator
    intersection = real_labels_matrix & predictions_labels_matrix
    #set union for binary is same as or operator
    union = real_labels_matrix | predictions_labels_matrix
    #sum(array.T) gets number of 1s in row
    row_wise_accuracy = sum(intersection.T) / sum(union.T)
    return sum(row_wise_accuracy) / real_labels_matrix.shape[0]

#size of intersection of predicted and actual labels divided by size of predicted set for each datapoint tested on
#sum those and divide by number of datapoints
#if no predicted labels, don't count that row towards the precision as that would be undefined
def multi_label_precision(real_labels_matrix, predictions_labels_matrix):
    #binary so set intersection is and operator
    intersection = real_labels_matrix & predictions_labels_matrix
    precision_sum = 0
    num_rows = 0
    for row in range(intersection.shape[0]):
        if sum(predictions_labels_matrix[row]) > 0: #if there is at least one prediction for this row
            num_rows += 1
            precision_sum += sum(intersection[row]) / sum(predictions_labels_matrix[row])
    if num_rows == 0:
        return 0#no labels predicted at all will give us 0 precision as precision makes no sense here
    return precision_sum / num_rows

#size of intersection of predicted and actual labels divided by size of real label set for each datapoint tested on
#sum those and divide by number of datapoints
#all datapoints should have at least 1 real label in this data set
#vectorized for speed
def multi_label_recall(real_labels_matrix, predictions_labels_matrix):
    #binary so set intersection is and operator
    intersection = real_labels_matrix & predictions_labels_matrix
    #set union for binary is same as or operator
    #sum(array.T) gets number of 1s in row
    row_wise_recall = sum(intersection.T) / sum(real_labels_matrix.T)
    return sum(row_wise_recall) / real_labels_matrix.shape[0]

#lower is better. Percent incorrectly chosen labels counting assignment and non-assignment equally
def hamming_loss(real_labels_matrix, predictions_labels_matrix):
    return (np.logical_xor(real_labels_matrix, predictions_labels_matrix)).sum()/(real_labels_matrix.shape[0] * real_labels_matrix.shape[1])


#K is what we imported keras backend as

#metric for keras for early stopping
#takes in raw labels from kerass (not yet converted to 0 and 1s)
#NOT the same as accuracy, this is total labels correctly identified divided by union of total labels
#this weights rows with more labels higher, where accruacy does not, but this is still a good metric for early stopping
def raw_multi_label_accuracy(y_true, y_pred):
    positives = K.greater_equal(y_pred, 0.5)
    positives = K.cast(positives, K.floatx())
    new_y_pred = positives #+ ((1-positives)*y_pred)
    intersection = y_true * new_y_pred
    union = 1 -((1-y_true)*(1-new_y_pred))
    accuracy = K.sum(intersection) / K.sum(union)
    return accuracy
    

In [21]:
def get_all_metrics(actual_labels, predictions):
    print('Getting evaluation metrics for each label:')
    get_per_label_metrics(actual_labels, predictions)
    print('Getting evaluations for multilabel problem')
    print('Multilabel accuracy: ' + str(multi_label_accuracy(actual_labels, predictions)))
    print('Multilabel precision: ' + str(multi_label_precision(actual_labels, predictions)))
    print('Multilabel recall: ' + str(multi_label_recall(actual_labels, predictions)))
    print("Percent of correctly decided label decisions: " + str(100* (1-hamming_loss(actual_labels, predictions))))

In [22]:
#for early stopping only after certain number of epochs. wait until delay epochs until early stopping
#not same as patience. Want to not even start looking until delay is reached
class DelayedEarlyStopping(EarlyStopping):
    def __init__(self, monitor, min_delta=0, patience=0, verbose=0, mode='auto', delay = 100):
        super(DelayedEarlyStopping, self).__init__(monitor=monitor, min_delta=min_delta, patience=patience,verbose=verbose, mode=mode)
        self.delay = delay

    def on_epoch_end(self, epoch, logs=None):
        if epoch > self.delay:
            super().on_epoch_end(epoch, logs)

In [23]:
def nn_output_to_predictions(res):
    label_predictions = []
    for i in range(res.shape[0]):
        pred = [0]*len(genre_dict)
        for j in range(res.shape[1]):
            if res[i][j] >= .5:
                pred[j] = 1
        label_predictions.append(pred)
    return np.array(label_predictions)

Convolutional Neural Networks

In [ ]:
model_cnn = Sequential()
e = Embedding(num_words_kept, word_vec_len, weights=[embedding_matrix], input_length=max_seq_len, trainable=True)
#e = Embedding(num_words_kept, word_vec_len, input_length=max_seq_len, trainable=True)
model_cnn.add(e)
model_cnn.add(Conv1D(filters=50, kernel_size=2, padding='valid', activation='relu', strides=1))
model_cnn.add(GlobalMaxPooling1D())
model_cnn.add(Dense(256, activation='relu', kernel_regularizer=l2(0.001)))
model_cnn.add(Dropout(.5))
model_cnn.add(Dense(len(genre_dict), activation='sigmoid'))
model_cnn.compile(loss='binary_crossentropy', optimizer='adam', metrics=[raw_multi_label_accuracy])
start = time.time()
model_cnn.fit(x_train_seq, y_train, validation_split = .1, callbacks = [DelayedEarlyStopping(monitor = 'val_raw_multi_label_accuracy', patience = 5, delay=25)], epochs=1000, batch_size=100, verbose=2)
end = time.time()
print('Time to train with cross validation for early stopping: ' + str(end-start) + ' seconds')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 2149 samples, validate on 239 samples
Epoch 1/1000
 - 3s - loss: 0.5787 - raw_multi_label_accuracy: 0.1347 - val_loss: 0.3989 - val_raw_multi_label_accuracy: 0.0000e+00
Epoch 2/1000
 - 2s - loss: 0.4067 - raw_multi_label_accuracy: 0.1152 - val_loss: 0.3722 - val_raw_multi_label_accuracy: 0.1697
Epoch 3/1000
 - 2s - loss: 0.3790 - raw_multi_label_accuracy: 0.1014 - val_loss: 0.3598 - val_raw_multi_label_accuracy: 0.1697
Epoch 4/1000
 - 2s - loss: 0.3645 - raw_multi_label_accuracy: 0.1092 - val_loss: 0.3510 - val_raw_multi_label_accuracy: 0.0000e+00
Epoch 5/1000
 - 2s - loss: 0.3545 - raw_multi_label_accuracy: 0.1081 - val_loss: 0.3447 - val_raw_multi_label_accuracy: 0.17

In [ ]:
predictions = nn_output_to_predictions(model_cnn.predict(x_test_seq))

In [ ]:
get_all_metrics(y_test, predictions)

Getting evaluation metrics for each label:
Accuruacy for War: 0.964824120603015
Precision for War: 0.0
Recall for War: 0.0

Accuruacy for Family: 0.9195979899497487
Precision for Family: 0.3333333333333333
Recall for Family: 0.043478260869565216

Accuruacy for Science Fiction: 0.8927973199329984
Precision for Science Fiction: 0.3333333333333333
Recall for Science Fiction: 0.03225806451612903

Accuruacy for Thriller: 0.6817420435510888
Precision for Thriller: 0.3870967741935484
Recall for Thriller: 0.21301775147928995

Accuruacy for Horror: 0.8961474036850922
Precision for Horror: 0.0
Recall for Horror: 0.0

Accuruacy for Romance: 0.7403685092127303
Precision for Romance: 0.2441860465116279
Recall for Romance: 0.1891891891891892

Accuruacy for Drama: 0.5862646566164154
Precision for Drama: 0.5833333333333334
Recall for Drama: 0.7899686520376176

Accuruacy for Foreign: 0.9882747068676717
Precision for Foreign: 0.0
Recall for Foreign: 0.0

Accuruacy for Documentary: 0.9731993299832495
Pre

/home/matt/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/matt/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)


CNN but with multiple filter sizes so we don't just filter on group of words at a time

In [ ]:
model_input = Input(shape=(max_seq_len,), dtype='int32')
e = Embedding(num_words_kept, word_vec_len, weights=[embedding_matrix], input_length=max_seq_len, trainable=True)(model_input)
two_word_filter = Conv1D(filters=100, kernel_size=2, padding='valid', activation='relu', strides=1)(e)
two_word_filter = GlobalMaxPooling1D()(two_word_filter)
three_word_filter = Conv1D(filters=100, kernel_size=3, padding='valid', activation='relu', strides=1)(e)
three_word_filter = GlobalMaxPooling1D()(three_word_filter)
four_word_filter = Conv1D(filters=100, kernel_size=4, padding='valid', activation='relu', strides=1)(e)
four_word_filter = GlobalMaxPooling1D()(four_word_filter)
merged = concatenate([two_word_filter, three_word_filter, four_word_filter], axis=1)

merged = Dense(256, activation='relu', kernel_regularizer=l2(0.01))(merged)
merged = Dropout(0.5)(merged)
merged = Dense(len(genre_dict))(merged)
output = Activation('sigmoid')(merged)
model = Model(inputs=[model_input], outputs=[output])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[raw_multi_label_accuracy])
start = time.time()
model.fit(x_train_seq, y_train, validation_split = .1, callbacks = [DelayedEarlyStopping(monitor = 'val_raw_multi_label_accuracy', patience = 5, delay=25)], epochs=1000, batch_size=100, verbose=2)
end = time.time()
print('Time to train with cross validation for early stopping: ' + str(end-start) + ' seconds')

Train on 2149 samples, validate on 239 samples
Epoch 1/1000
 - 6s - loss: 2.6106 - raw_multi_label_accuracy: 0.1168 - val_loss: 1.8744 - val_raw_multi_label_accuracy: 0.1481
Epoch 2/1000
 - 6s - loss: 1.4934 - raw_multi_label_accuracy: 0.1052 - val_loss: 1.0871 - val_raw_multi_label_accuracy: 0.1688
Epoch 3/1000
 - 5s - loss: 0.8868 - raw_multi_label_accuracy: 0.1168 - val_loss: 0.6795 - val_raw_multi_label_accuracy: 0.1697
Epoch 4/1000
 - 5s - loss: 0.5843 - raw_multi_label_accuracy: 0.1017 - val_loss: 0.4814 - val_raw_multi_label_accuracy: 0.1590
Epoch 5/1000
 - 5s - loss: 0.4405 - raw_multi_label_accuracy: 0.0906 - val_loss: 0.3920 - val_raw_multi_label_accuracy: 0.1274
Epoch 6/1000
 - 6s - loss: 0.3765 - raw_multi_label_accuracy: 0.1054 - val_loss: 0.3528 - val_raw_multi_label_accuracy: 0.1697
Epoch 7/1000
 - 5s - loss: 0.3468 - raw_multi_label_accuracy: 0.0976 - val_loss: 0.3330 - val_raw_multi_label_accuracy: 0.0224
Epoch 8/1000
 - 5s - loss: 0.3332 - raw_multi_label_accuracy: 0.

In [ ]:
predictions = nn_output_to_predictions(model.predict(x_test_seq))

In [ ]:
get_all_metrics(y_test, predictions)

Getting evaluation metrics for each label:
Accuruacy for War: 0.964824120603015
Precision for War: 0.0
Recall for War: 0.0

Accuruacy for Family: 0.9229480737018425
Precision for Family: 0.0
Recall for Family: 0.0

Accuruacy for Science Fiction: 0.8944723618090452
Precision for Science Fiction: 0.0
Recall for Science Fiction: 0.0

Accuruacy for Thriller: 0.6884422110552764
Precision for Thriller: 0.4105263157894737
Recall for Thriller: 0.23076923076923078

Accuruacy for Horror: 0.9061976549413735
Precision for Horror: 0.0
Recall for Horror: 0.0

Accuruacy for Romance: 0.8123953098827471
Precision for Romance: 0.4782608695652174
Recall for Romance: 0.0990990990990991

Accuruacy for Drama: 0.5678391959798995
Precision for Drama: 0.5927051671732523
Recall for Drama: 0.6112852664576802

Accuruacy for Foreign: 0.9882747068676717
Precision for Foreign: 0.0
Recall for Foreign: 0.0

Accuruacy for Documentary: 0.9731993299832495
Precision for Documentary: 0.0
Recall for Documentary: 0.0

Accuru

Regular Neural Network

In [ ]:
normal_nn = Sequential()
e = Embedding(num_words_kept, word_vec_len, weights=[embedding_matrix], input_length=max_seq_len, trainable=True)
normal_nn.add(e)
normal_nn.add(Flatten())
normal_nn.add(Dense(256, activation='relu'))
normal_nn.add(Dense(len(genre_dict), activation='sigmoid'))
normal_nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=[raw_multi_label_accuracy])
start = time.time()
normal_nn.fit(x_train_seq, y_train, validation_split = .1, callbacks = [DelayedEarlyStopping(monitor = 'val_raw_multi_label_accuracy', patience = 5, delay=25)], epochs=1000, batch_size=100, verbose=2)
end = time.time()
print('Time to train with cross validation for early stopping: ' + str(end-start) + ' seconds')

Train on 2149 samples, validate on 239 samples
Epoch 1/1000
 - 3s - loss: 0.4020 - raw_multi_label_accuracy: 0.1224 - val_loss: 0.3433 - val_raw_multi_label_accuracy: 0.0373
Epoch 2/1000
 - 2s - loss: 0.3228 - raw_multi_label_accuracy: 0.1162 - val_loss: 0.3297 - val_raw_multi_label_accuracy: 0.1159
Epoch 3/1000
 - 2s - loss: 0.3062 - raw_multi_label_accuracy: 0.1260 - val_loss: 0.3284 - val_raw_multi_label_accuracy: 0.0363
Epoch 4/1000
 - 2s - loss: 0.2952 - raw_multi_label_accuracy: 0.1330 - val_loss: 0.3218 - val_raw_multi_label_accuracy: 0.1126
Epoch 5/1000
 - 2s - loss: 0.2852 - raw_multi_label_accuracy: 0.1635 - val_loss: 0.3218 - val_raw_multi_label_accuracy: 0.1571
Epoch 6/1000
 - 2s - loss: 0.2697 - raw_multi_label_accuracy: 0.2038 - val_loss: 0.3242 - val_raw_multi_label_accuracy: 0.1445
Epoch 7/1000
 - 2s - loss: 0.2491 - raw_multi_label_accuracy: 0.2592 - val_loss: 0.3191 - val_raw_multi_label_accuracy: 0.1517
Epoch 8/1000
 - 2s - loss: 0.2235 - raw_multi_label_accuracy: 0.

In [ ]:
predictions = nn_output_to_predictions(normal_nn.predict(x_test_seq))

In [ ]:
get_all_metrics(y_test, predictions)

Getting evaluation metrics for each label:
Accuruacy for War: 0.964824120603015
Precision for War: 0.0
Recall for War: 0.0

Accuruacy for Family: 0.9229480737018425
Precision for Family: 0.0
Recall for Family: 0.0

Accuruacy for Science Fiction: 0.9028475711892797
Precision for Science Fiction: 1.0
Recall for Science Fiction: 0.06451612903225806

Accuruacy for Thriller: 0.7085427135678392
Precision for Thriller: 0.4358974358974359
Recall for Thriller: 0.10059171597633136

Accuruacy for Horror: 0.9045226130653267
Precision for Horror: 0.3333333333333333
Recall for Horror: 0.017857142857142856

Accuruacy for Romance: 0.8257956448911222
Precision for Romance: 0.5897435897435898
Recall for Romance: 0.2072072072072072

Accuruacy for Drama: 0.5845896147403685
Precision for Drama: 0.6011396011396012
Recall for Drama: 0.6614420062695925

Accuruacy for Foreign: 0.9882747068676717
Precision for Foreign: 0.0
Recall for Foreign: 0.0

Accuruacy for Documentary: 0.9731993299832495
Precision for Docu

LSTM

In [ ]:
lstm_model = Sequential()
e = Embedding(num_words_kept, word_vec_len, weights=[embedding_matrix], input_length=max_seq_len, trainable=True)
lstm_model.add(e)
lstm_model.add(LSTM(100, dropout=0.25, recurrent_dropout=0.25))
lstm_model.add(Dense(256, activation='relu'))
lstm_model.add(Dense(len(genre_dict), activation='sigmoid'))
lstm_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[raw_multi_label_accuracy])
start = time.time()
lstm_model.fit(x_train_seq, y_train, validation_split = .1, callbacks = [DelayedEarlyStopping(monitor = 'val_raw_multi_label_accuracy', patience = 5, delay=25)], epochs=1000, batch_size=100, verbose=2)
end = time.time()
print('Time to train with cross validation for early stopping: ' + str(end-start) + ' seconds')

Train on 2149 samples, validate on 239 samples
Epoch 1/1000
 - 25s - loss: 0.5003 - raw_multi_label_accuracy: 0.1572 - val_loss: 0.3284 - val_raw_multi_label_accuracy: 0.0000e+00
Epoch 2/1000
 - 10s - loss: 0.3181 - raw_multi_label_accuracy: 0.0866 - val_loss: 0.3164 - val_raw_multi_label_accuracy: 0.1695
Epoch 3/1000
 - 9s - loss: 0.3128 - raw_multi_label_accuracy: 0.0921 - val_loss: 0.3155 - val_raw_multi_label_accuracy: 0.0160
Epoch 4/1000
 - 9s - loss: 0.3123 - raw_multi_label_accuracy: 0.1275 - val_loss: 0.3142 - val_raw_multi_label_accuracy: 0.0553
Epoch 5/1000
 - 10s - loss: 0.3120 - raw_multi_label_accuracy: 0.0890 - val_loss: 0.3144 - val_raw_multi_label_accuracy: 0.0816
Epoch 6/1000
 - 12s - loss: 0.3117 - raw_multi_label_accuracy: 0.0857 - val_loss: 0.3146 - val_raw_multi_label_accuracy: 0.0127
Epoch 7/1000
 - 12s - loss: 0.3118 - raw_multi_label_accuracy: 0.1395 - val_loss: 0.3143 - val_raw_multi_label_accuracy: 0.0322
Epoch 8/1000
 - 9s - loss: 0.3113 - raw_multi_label_acc

In [ ]:
predictions = nn_output_to_predictions(lstm_model.predict(x_test_seq))

In [ ]:
get_all_metrics(y_test, predictions)

Getting evaluation metrics for each label:
Accuruacy for War: 0.964824120603015
Precision for War: 0.0
Recall for War: 0.0

Accuruacy for Family: 0.916247906197655
Precision for Family: 0.25
Recall for Family: 0.043478260869565216

Accuruacy for Science Fiction: 0.8760469011725294
Precision for Science Fiction: 0.3235294117647059
Recall for Science Fiction: 0.1774193548387097

Accuruacy for Thriller: 0.695142378559464
Precision for Thriller: 0.4496124031007752
Recall for Thriller: 0.3431952662721893

Accuruacy for Horror: 0.8777219430485762
Precision for Horror: 0.2702702702702703
Recall for Horror: 0.17857142857142858

Accuruacy for Romance: 0.7889447236180904
Precision for Romance: 0.4117647058823529
Recall for Romance: 0.3153153153153153

Accuruacy for Drama: 0.5745393634840871
Precision for Drama: 0.599388379204893
Recall for Drama: 0.6144200626959248

Accuruacy for Foreign: 0.9882747068676717
Precision for Foreign: 0.0
Recall for Foreign: 0.0

Accuruacy for Documentary: 0.96984924

simple rnn

In [ ]:
rnn = Sequential()
e = Embedding(num_words_kept, word_vec_len, weights=[embedding_matrix], input_length=max_seq_len, trainable=True)
rnn.add(e)
rnn.add(SimpleRNN(32, activation = 'relu'))
rnn.add(Dense(256, activation='relu'))
rnn.add(Dense(len(genre_dict), activation='sigmoid'))
rnn.compile(loss='binary_crossentropy', optimizer='adam', metrics=[raw_multi_label_accuracy])
start = time.time()
rnn.fit(x_train_seq, y_train, validation_split = .1, callbacks = [DelayedEarlyStopping(monitor = 'val_raw_multi_label_accuracy', patience = 5, delay=25)], epochs=1000, batch_size=100, verbose=2)
end = time.time()
print('Time to train with cross validation for early stopping: ' + str(end-start) + ' seconds')

Train on 2149 samples, validate on 239 samples
Epoch 1/1000
 - 3s - loss: 0.5408 - raw_multi_label_accuracy: 0.0644 - val_loss: 0.3440 - val_raw_multi_label_accuracy: 0.1695
Epoch 2/1000
 - 2s - loss: 0.3323 - raw_multi_label_accuracy: 0.1113 - val_loss: 0.3253 - val_raw_multi_label_accuracy: 0.1682
Epoch 3/1000
 - 1s - loss: 0.3197 - raw_multi_label_accuracy: 0.1197 - val_loss: 0.3206 - val_raw_multi_label_accuracy: 0.0845
Epoch 4/1000
 - 2s - loss: 0.3126 - raw_multi_label_accuracy: 0.1305 - val_loss: 0.3158 - val_raw_multi_label_accuracy: 0.1429
Epoch 5/1000
 - 2s - loss: 0.3054 - raw_multi_label_accuracy: 0.1205 - val_loss: 0.3126 - val_raw_multi_label_accuracy: 0.1624
Epoch 6/1000
 - 2s - loss: 0.2985 - raw_multi_label_accuracy: 0.1397 - val_loss: 0.3110 - val_raw_multi_label_accuracy: 0.0888
Epoch 7/1000
 - 2s - loss: 0.2909 - raw_multi_label_accuracy: 0.1392 - val_loss: 0.3097 - val_raw_multi_label_accuracy: 0.1399
Epoch 8/1000


In [ ]:
predictions = nn_output_to_predictions(rnn.predict(x_test_seq))

In [ ]:
get_all_metrics(y_test, predictions)

bidirectional-LSTM

In [ ]:
bi_lstm = Sequential()
e = Embedding(num_words_kept, word_vec_len, weights=[embedding_matrix], input_length=max_seq_len, trainable=True)
bi_lstm.add(e)
bi_lstm.add(Bidirectional(LSTM(100, dropout=0.25, recurrent_dropout=0.25)))
bi_lstm.add(Dense(256, activation='relu'))
bi_lstm.add(Dense(len(genre_dict), activation='sigmoid'))
bi_lstm.compile(loss='binary_crossentropy', optimizer='adam', metrics=[raw_multi_label_accuracy])
start = time.time()
bi_lstm.fit(x_train_seq, y_train, validation_split = .1, callbacks = [DelayedEarlyStopping(monitor = 'val_raw_multi_label_accuracy', patience = 5, delay=25)], epochs=1000, batch_size=100, verbose=2)
end = time.time()
print('Time to train with cross validation for early stopping: ' + str(end-start) + ' seconds')

In [ ]:
predictions = nn_output_to_predictions(bi_lstm.predict(x_test_seq))

In [ ]:
get_all_metrics(y_test, predictions)